<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [5]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [7]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [8]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [9]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [10]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [11]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [12]:
import dagshub
dagshub.init(repo_owner='srinathranganathan432', repo_name='ML_flow', mlflow=True)

Accessing as srinathranganathan432

Initialized MLflow to track repo "srinathranganathan432/ML_flow"

Repository srinathranganathan432/ML_flow initialized!

In [13]:
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = 'srinathranganathan432' # 'learnpythonlanguage'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'b81460488c629970a384e0520ee58535a6af29bf' # 
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/srinathranganathan432/ML_flow.mlflow' # https://dagshub.com/learnpythonlanguage/mlflow_dagshub_demo.mlflow


In [15]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
# mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_tracking_uri("https://dagshub.com/srinathranganathan432/ML_flow.mlflow")
for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/10/18 21:48:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/18 21:48:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: https://dagshub.com/srinathranganathan432/ML_flow.mlflow/#/experiments/0/runs/c7f0dee28e3246e59df55e5cff9604bb.
2024/10/18 21:48:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/srinathranganathan432/ML_flow.mlflow/#/experiments/0.
2024/10/18 21:48:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/18 21:48:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: https://dagshub.com/srinathranganathan432/ML_flow.mlflow/#/experiments/0/runs/fbfaa1709fc044c4a87fcbaa443c17eb.
2024/10/18 21:48:27 INF

### Register the Model

In [16]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please type RunID 9b948cef88ee468aaedb8a5dc1da90d3


Successfully registered model 'XGB-Smote'.
2024/10/18 21:50:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.
2024/10/18 21:50:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/srinathranganathan432/ML_flow.mlflow/#/experiments/0/runs/9b948cef88ee468aaedb8a5dc1da90d3.
2024/10/18 21:50:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/srinathranganathan432/ML_flow.mlflow/#/experiments/0.


### Load the Model

In [18]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production

In [29]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '4' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1729265397841, current_stage='None', description='', last_updated_timestamp=1729265397841, name='anomaly-detection-prod', run_id='ef77e7a3710a40a3a2578edcd7f9917c', run_link='', source='models:/XGB-Smote/4', status='READY', status_message='', tags={}, user_id='', version='1'>

In [33]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 